In [2]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta
import concurrent.futures
import sys
import os.path
import bson as b


from pyspark.sql.types import *

from pyspark.sql import types as T 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta 
import pyspark
from pyspark.sql import SQLContext  

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = (SparkConf()\
    .setMaster('local')\
    .setAppName('sparkApp')\
    .set('spark.Builder.enableHiveSupport',True)\
    .set("spark.sql.session.timeZone", "America/Sao_Paulo")\
    .set('spark.sql.execution.arrow.pyspark.enabled' , False)\
    .set('spark.sql.caseSensitive', True))
sc = SparkContext(conf=conf) 
 
spark = SparkSession(sc)

uri_string = "mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false"
dbName = 'local'
colName = 'STP'
client = MongoClient(uri_string)
database = client.get_database(dbName)
col = database.get_collection(colName)


#PYMONGO METHODS

In [3]:

def colCount():
    colCount =  col.find({},{"_id"}).count()
    return colCount

def colFind(limit=""):
    if limit == "":
        colList = [ x for x in col.find()]
    else:
        colList = [ x for x in col.find().limit(limit)]
    return(colList)

def colRawFind(limit=""):
    if limit == "":
        colList = [ x for x in col.find_raw_batches()]
    else:
        colList = [ b.decode_all(x) for x in col.find_raw_batches().limit(limit)]
    return(colList)

def colFindID():
    idList = [str(x).split("'",4)[3] for x in col.find({},{"_id":1}).limit(5)]
    return(idList)
    


#PYSPARK METHODS

In [1]:
DATA_SCHEMA = spark.read\
                .format( "com.mongodb.spark.sql.DefaultSource")\
                .option('spark.mongodb.input.sampleSize', 100)\
                .option("database", "loca")\
                .option("spark.mongodb.input.collection", "STP")\
                .load().schema


NameError: name 'spark' is not defined

#PARALLEL METHODS

In [ ]:
def maxRunWorkerTask(ID=None):
    #TRANSFORMATION FUNC

    QUERY = db.client.get_database("local").get_collection("SPT").find({ 'match' : {'_id': ID}})     
    for x in QUERY:
        RESULT_LIST.append(x)
    

def maxRunWorker(max_workers=4, mode="submit"):    
    ID_LIST = colFindID()
    T0 = datetime.now()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        if mode == "submit":
            for x in ID_LIST:
                executor.submit(maxRunWorkerTask, ID=x)
        else:
            executor.map(maxRunWorkerTask, ID_LIST)
        
    T1 = datetime.now()
    print(f" QUERY_TIME = {(T1-T0).seconds} seconds")
